# Otomatik Metrikler: BLEU, ROUGE, Perplexity (Örnekler ve Açıklama)
Bu notebook BLEU, ROUGE ve Perplexity metriklerini Türkçe olarak açıklar ve her biri için çalıştırılabilir Python örnekleri içerir. Hücreleri sırayla çalıştırarak kendi verinizle deneme yapabilirsiniz.

## 1) BLEU
- Ne ölçer: Üretilen (candidate) metin ile bir veya daha fazla referans metin arasındaki n-gram örtüşmesini ve kısa çıktılara ceza (brevity penalty) uygular.
- Temel formül (özet): $$BLEU = BP \\cdot \\exp\\left(\\sum_{n=1}^N w_n \\log p_n\\right)$$
  - p_n: kırpılmış n-gram doğruluğu, w_n: ağırlıklar (çoğunlukla eşit), BP: brevity penalty.
- Kullanım: Makine çevirisi ve generation görevlerinde sık kullanılır; kısa cümlelerde veya tek referanslı durumlarda gürültülü olabilir.

In [1]:
# BLEU örneği (sacrebleu)
# Not: İlk çalıştırmada paketleri yükleyin. Aşağıdaki satırı çalıştırabilirsiniz:
!pip install -q sacrebleu
from sacrebleu import corpus_bleu

# İki örnek candidate ve iki referanslı yapı
candidates = [
    "The cat is on the mat",
    "There is a cat"
]
# sacrebleu için referanslar: her referansın tüm candidate'lar için listesi
ref1 = [
    "The cat is on the mat",
    "There is a cat on the mat"
]
ref2 = [
    "The cat sits on the mat",
    "A cat exists"
]
bleu = corpus_bleu(candidates, [ref1, ref2])
print(f"BLEU score: {bleu.score:.2f}")


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
BLEU score: 100.00
BLEU score: 100.00


## 2) ROUGE
- Ne ölçer: Özellikle özetleme için uygundur; referans özetteki n-gram'lerin (çoğunlukla recall odaklı) yakalanıp yakalanmadığını gösterir. En yaygınler: ROUGE-1, ROUGE-2, ROUGE-L.
- Kullanım: Genelde özetleme görevlerinde, birden çok referans varsa daha güvenilir skor verir.

In [2]:
# ROUGE örneği (huggingface evaluate)
!pip install -q evaluate
import evaluate
rouge = evaluate.load("rouge")

predictions = [
    "the cat is on the mat",
    "there is a cat on the mat"
]
references = [
    "the cat sits on the mat",
    "a cat is on the mat"
]
results = rouge.compute(predictions=predictions, references=references)
print(results)


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


/home/mehmetcelik/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 

## 3) Perplexity (PPL)
- Ne ölçer: Bir dil modelinin bir metni ne kadar iyi tahmin ettiğinin ölçüsüdür. Daha düşük perplexity daha iyi.
- Formül (özet): $$Perplexity = \\exp\\left(-\\frac{1}{N}\\sum_{t=1}^N \\log p(x_t\\mid x_{<t})\\right)$$
- Kullanım: Modelin genel dil yeteneğini ölçmede faydalıdır; model ve tokenizasyon bağımlıdır, bu yüzden sadece aynı tokenizer/model karşılaştırmalarında anlamlıdır.

In [ ]:
# Perplexity örneği (transformers ile, GPT-2) -- güvenli sürüm
# Not: Ağır paket kurulumları veya model indirmeleri kernel'i çökertirse, paketleri terminalde kurup kernel'i yeniden başlatın.
try:
    import torch
    from transformers import AutoTokenizer, AutoModelForCausalLM
except Exception as e:
    print('transformers veya torch import edilirken hata:', e)
    print()
    print('Çözüm: terminalde aşağıdaki komutu çalıştırın ve sonra Jupyter kernel\'ini yeniden başlatın:')
    print()
    print('    pip install transformers torch')
    print()
    print('Not: Büyük modeller indirilebilir; eğer model indirmek istemiyorsanız aşağıdaki basit (örnek olasılık) hesaplamasını kullanın.')
else:
    import math
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model_name = 'gpt2'
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    model.eval()

    def perplexity_of_text(text):
        enc = tokenizer(text, return_tensors='pt')
        input_ids = enc.input_ids.to(device)
        with torch.no_grad():
            outputs = model(input_ids, labels=input_ids)
            loss = outputs.loss.item()  # average cross-entropy per token
        ppl = math.exp(loss)
        return loss, ppl

    text = 'The quick brown fox jumps over the lazy dog.'
    loss, ppl = perplexity_of_text(text)
    print(f'Loss (cross-entropy): {loss:.4f}, Perplexity: {ppl:.2f}')

# Alternatif: eğer model indirmek/kurmak istemiyorsanız bu basit örneği çalıştırın
# (gerçek model olasılıkları yerine örnek olasılıklar kullanır ve formülü gösterir)
probs = [0.1, 0.01, 0.5, 0.2]
import math
n = len(probs)
cross_entropy = - sum(math.log(p) for p in probs) / n
perplexity = math.exp(cross_entropy)
print('Basit örnek perplexity (örnek olasılıklar):', perplexity)


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


/home/mehmetcelik/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 

## Notlar ve İpuçları
- BLEU ve ROUGE sonuçları tokenizasyona çok bağımlıdır; aynı tokenizasyonu kullanın.
- BLEU için smoothing kısa cümlelerde önemlidir. NLTK'de SmoothingFunction kullanılabilir.
- Perplexity sadece aynı model/tokenizer arasında karşılaştırılmalıdır.
- Daha semantik ölçümler için BERTScore, BLEURT veya embedding tabanlı metrikleri düşünebilirsiniz.